In [1]:
from google.colab import drive
from google.colab import files


drive.mount('/content/gdrive')
!ls

Mounted at /content/gdrive
gdrive	sample_data


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense
from tensorflow.keras import Input, Model
from tensorflow.keras import backend as K, Model, Input, optimizers
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation, SpatialDropout1D, Lambda
from tensorflow.keras.layers import Layer, Conv1D, MaxPooling1D, Dense, BatchNormalization, LayerNormalization
import inspect
from tensorflow.keras.utils import to_categorical

class5 = {0:'Exploit Kit', 1:'malspam', 2:'Phishing', 3:'Ransomware', 4:'Trojan'}
learning_rate = 0.002  
filters_num = 16
filter_size = 8
TRAIN_EPOCHS = 100
dilation_num = 9

def update_confusion_matrix(confusion_matrix, actual_lb, predict_lb):
    for idx, value in enumerate(actual_lb):
        p_value = predict_lb[idx]
        confusion_matrix[value, p_value] += 1
    return confusion_matrix

def truncate(f, n):
    trunc_f = np.math.floor(f * 10 ** n) / 10 ** n
    return '{:.4f}'.format(trunc_f) # only for 0.0 => 0.0000

def get_data():
    print('[*] load data')
    #os.getcwd()         # check current directory
    with tf.device('/GPU:0'):
      data = np.loadtxt('/content/gdrive/My Drive/tcn/train_final.csv', delimiter=",", dtype=np.float32)
      test = np.loadtxt('/content/gdrive/My Drive/tcn/test_final.csv', delimiter=",", dtype=np.float32)
      print('[-] done!')
      tr_x = data[:,:-1]
      te_x = test[:,:-1]
      #print(tr_x.shape)
      tr_x = np.expand_dims(tr_x, axis=2)
      te_x = np.expand_dims(te_x, axis=2)
    
      tr_y = data[:,-1]
      tr_y = to_categorical(tr_y, 5)   # one-hot encoding
      tr_y = np.expand_dims(tr_y, axis=2)
      te_y = test[:,-1]
      te_y = to_categorical(te_y, 5)   # one-hot encoding
      te_y = np.expand_dims(te_y, axis=2)
    #print(len(data))   # 2400
  
    return tr_x, tr_y, te_x, te_y


def is_power_of_two(num):     # return : True or False
    return num != 0 and ((num & (num - 1)) == 0)


def adjust_dilations(dilations):   # if dilataions == True return dilations 
    if all([is_power_of_two(i) for i in dilations]): 
        return dilations
    else:
        new_dilations = [2 ** i for i in dilations]
        return new_dilations



In [3]:
class ResidualBlock(Layer):

    def __init__(self,
                 dilation_rate,                            # dilation power of 2
                 nb_filters,                               # number of conv filters
                 kernel_size,                              # conv kernel size
                 padding,                                  # valid(no padding), same(inputlen=outputlen), causal
                 activation='relu',                        # o = Activation(x + F(x))
                 dropout_rate=0,                           # [0, 1]
                 kernel_initializer='he_normal',      
                 use_batch_norm=True,                     
                 use_layer_norm=False,
                 last_block=True,
                 **kwargs):                                # key, value 


        self.dilation_rate = dilation_rate
        self.nb_filters = nb_filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.activation = activation
        self.dropout_rate = dropout_rate
        self.kernel_initializer = kernel_initializer
        self.use_batch_norm = use_batch_norm
        self.use_layer_norm = use_layer_norm
        self.last_block = last_block
        
        self.layers = []
        self.layers_outputs = []
        self.shape_match_conv = None
        self.res_output_shape = None
        self.final_activation = None

        super(ResidualBlock, self).__init__(**kwargs)

    def _add_and_activate_layer(self, layer):               # append layer to internal layer list
        self.layers.append(layer)
        self.layers[-1].build(self.res_output_shape)
        self.res_output_shape = self.layers[-1].compute_output_shape(self.res_output_shape)

    def build(self, input_shape):

        with K.name_scope(self.name):  # name scope used to make sure weights get unique names
            self.layers = []
            self.res_output_shape = input_shape

            for k in range(2):
                with K.name_scope('conv1D_{}'.format(k)):         # conv1D_0 or conv1D_1
                    self._add_and_activate_layer(Conv1D(filters=self.nb_filters,
                                                        kernel_size=self.kernel_size,
                                                        dilation_rate=self.dilation_rate,
                                                        padding=self.padding,
                                                        name='conv1D_{}'.format(k),
                                                        kernel_initializer=self.kernel_initializer))
                    #self._add_and_activate_layer(MaxPooling1D(pool_size=3))

                with K.name_scope('norm_{}'.format(k)):           # norm_0 or norm_1
                    if self.use_batch_norm:
                        self._add_and_activate_layer(BatchNormalization())
                        print('use batch_norm')
                    elif self.use_layer_norm:
                        self._add_and_activate_layer(LayerNormalization())
                        print('use layer_norm')

                self._add_and_activate_layer(Activation('relu'))
                self._add_and_activate_layer(SpatialDropout1D(rate=self.dropout_rate))

            if not self.last_block:
                # 1x1 conv to match the shapes (channel dimension).
                name = 'conv1D_{}'.format(k + 1)
                with K.name_scope(name):
                    # make and build this layer separately because it directly uses input_shape
                    self.shape_match_conv = Conv1D(filters=self.nb_filters,
                                                   kernel_size=1,
                                                   padding='same',            # len(input) == len(output)
                                                   name=name,
                                                   kernel_initializer=self.kernel_initializer)

            else:
                self.shape_match_conv = Lambda(lambda x: x, name='identity')   # Lambda : Layer로 감싸줌

            self.shape_match_conv.build(input_shape)
            self.res_output_shape = self.shape_match_conv.compute_output_shape(input_shape)

            self.final_activation = Activation(self.activation)
            self.final_activation.build(self.res_output_shape)  # probably isn't necessary

            # this is done to force Keras to add the layers in the list to self._layers
            for layer in self.layers:
                self.__setattr__(layer.name, layer)

            super(ResidualBlock, self).build(input_shape)  # done to make sure self.built is set True

    def call(self, inputs, training=None):                 # training : boolean whether it's training mode or not
        x = inputs                           
        self.layers_outputs = [x]
        for layer in self.layers:
            training_flag = 'training' in dict(inspect.signature(layer.call).parameters)
            x = layer(x, training=training) if training_flag else layer(x)
            # training_flag == False -> x = layer(x)
            self.layers_outputs.append(x)
        x2 = self.shape_match_conv(inputs)
        self.layers_outputs.append(x2)
        res_x = layers.add([x2, x])
        self.layers_outputs.append(res_x)

        res_act_x = self.final_activation(res_x)            
        self.layers_outputs.append(res_act_x)
        return [res_act_x, x]                               # residual model tensor, skip connection tensor

    def compute_output_shape(self, input_shape):
        return [self.res_output_shape, self.res_output_shape]

In [4]:
class TCN(Layer):              # input shape (batch_size, timesteps, input_dim), return TCN layer
    def __init__(self,
                 nb_filters=64,
                 kernel_size=16,
                 nb_stacks=1,
                 dilations=(1, 2, 4, 8, 16, 32),
                 padding='causal',
                 use_skip_connections=True,
                 dropout_rate=0.0,
                 return_sequences=False,
                 activation='linear',
                 kernel_initializer='he_normal',
                 use_batch_norm=False,
                 use_layer_norm=False,
                 **kwargs):                        # to configure parent class layer
        #print('__init__\n')
        self.return_sequences = return_sequences
        self.dropout_rate = dropout_rate
        self.use_skip_connections = use_skip_connections
        self.dilations = dilations
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.nb_filters = nb_filters
        self.activation = activation
        self.padding = padding
        self.kernel_initializer = kernel_initializer
        self.use_batch_norm = use_batch_norm
        self.use_layer_norm = use_layer_norm
        self.skip_connections = []
        self.residual_blocks = []
        self.layers_outputs = []
        self.main_conv1D = None
        self.build_output_shape = None
        self.lambda_layer = None
        self.lambda_ouput_shape = None

        if padding != 'causal' and padding != 'same':   # Must use padding
            raise ValueError("Only 'causal' or 'same' padding are compatible for this layer.")

        if not isinstance(nb_filters, int):          # check whether nb_filters(64) is integer or not
            print('An interface change occurred after the version 2.1.2.')
            print('Before: tcn.TCN(x, return_sequences=False, ...)')
            print('Now should be: tcn.TCN(return_sequences=False, ...)(x)')
            print('The alternative is to downgrade to 2.1.2 (pip install keras-tcn==2.1.2).')
            raise Exception()

        # initialize parent class
        super(TCN, self).__init__(**kwargs)

    @property     # getter
    def receptive_field(self):
        assert_msg = 'The receptive field formula works only with power of two dilations.'
        assert all([is_power_of_two(i) for i in self.dilations]), assert_msg
        # check dilation numbers are power of two or not / if not true, raise assert error
        return self.kernel_size * self.nb_stacks * self.dilations[-1]

    def build(self, input_shape):
        #print('build\n')
        
        self.main_conv1D = Conv1D(filters=64,#self.nb_filters,    # of filters
                                  kernel_size=10,              # same with Dense layer
                                  padding=self.padding,       # causal
                                  kernel_initializer=self.kernel_initializer)  # he_normal
        '''
        block = Conv1D(filters=64,#self.nb_filters,    # of filters
                       kernel_size=10,              # same with Dense layer
                       padding=self.padding,       # causal
                       kernel_initializer=self.kernel_initializer)  # he_normal
        block = MaxPooling1D(pool_size=3)(block)
        self.main_conv1D = block
        '''
        self.main_conv1D.build(input_shape)

        # member to hold current output shape of the layer for building purposes
        self.build_output_shape = self.main_conv1D.compute_output_shape(input_shape)

        # list to hold all the member ResidualBlocks
        self.residual_blocks = []
        total_num_blocks = self.nb_stacks * len(self.dilations)     # block = layer, total_num_blocks = 6
        if not self.use_skip_connections:
            total_num_blocks += 1  # cheap way to do a false case for below

        for s in range(self.nb_stacks):
            for d in self.dilations:
                self.residual_blocks.append(ResidualBlock(dilation_rate=d,
                                                          nb_filters=self.nb_filters,
                                                          kernel_size=self.kernel_size,
                                                          padding=self.padding,
                                                          activation=self.activation,
                                                          dropout_rate=self.dropout_rate,
                                                          use_batch_norm=self.use_batch_norm,
                                                          use_layer_norm=self.use_layer_norm,
                                                          kernel_initializer=self.kernel_initializer,
                                                          last_block=len(self.residual_blocks) + 1 == total_num_blocks,
                                                          name='residual_block_{}'.format(len(self.residual_blocks))))
                # build newest residual block
                self.residual_blocks[-1].build(self.build_output_shape)
                self.build_output_shape = self.residual_blocks[-1].res_output_shape

        # this is done to force keras to add the layers in the list to self._layers
        for layer in self.residual_blocks:
            self.__setattr__(layer.name, layer)

        # Author: @karolbadowski.
        output_slice_index = int(self.build_output_shape.as_list()[1] / 2) if self.padding == 'same' else -1
        #print(self.build_output_shape.as_list())   #(None, 10000, nb_filters)
        #print('output slice index : ', output_slice_index)   # -1
        self.lambda_layer = Lambda(lambda tt: tt[:, output_slice_index, :])
        self.lambda_ouput_shape = self.lambda_layer.compute_output_shape(self.build_output_shape)

    def compute_output_shape(self, input_shape):
        if not self.built:
            self.build(input_shape)
        if not self.return_sequences:
            return self.lambda_ouput_shape
        else:
            return self.build_output_shape

    def call(self, inputs, training=None):
        #print('call\n')
        x = inputs
        self.layers_outputs = [x]
        try:
            x = self.main_conv1D(x)
            self.layers_outputs.append(x)
        except AttributeError:
            print('The backend of keras-tcn>2.8.3 has changed from keras to tensorflow.keras.')
            print('Either update your imports:\n- From "from keras.layers import <LayerName>" '
                  '\n- To "from tensorflow.keras.layers import <LayerName>"')
            print('Or downgrade to 2.8.3 by running "pip install keras-tcn==2.8.3"')
            import sys
            sys.exit(0)
        self.skip_connections = []
        for layer in self.residual_blocks:
            x, skip_out = layer(x, training=training)
            self.skip_connections.append(skip_out)
            self.layers_outputs.append(x)

        if self.use_skip_connections:
            x = layers.add(self.skip_connections)
            self.layers_outputs.append(x)
        if not self.return_sequences:
            x = self.lambda_layer(x)
            self.layers_outputs.append(x)
        return x

    def get_config(self):
        config = super(TCN, self).get_config()
        config['nb_filters'] = self.nb_filters
        config['kernel_size'] = self.kernel_size
        config['nb_stacks'] = self.nb_stacks
        config['dilations'] = self.dilations
        config['padding'] = self.padding
        config['use_skip_connections'] = self.use_skip_connections
        config['dropout_rate'] = self.dropout_rate
        config['return_sequences'] = self.return_sequences
        config['activation'] = self.activation
        config['use_batch_norm'] = self.use_batch_norm
        config['use_layer_norm'] = self.use_layer_norm
        config['kernel_initializer'] = self.kernel_initializer
        return config

In [5]:
def compiled_tcn(num_feat,  # type: int
                 num_classes,  # type: int                size of final dense layer(# of classes)
                 nb_filters,  # type: int                 
                 kernel_size,  # type: int
                 dilations,  # type: List[int]
                 nb_stacks,  # type: int
                 max_len,  # type: int                    max sequence length
                 output_len=1,  # type: int
                 padding='causal',  # type: str
                 use_skip_connections=True,  # type: bool
                 return_sequences=True,
                 regression=False,  # type: bool          continuous or discrete
                 dropout_rate=0.05,  # type: float
                 name='tcn',  # type: str,
                 kernel_initializer='he_normal',  # type: str,
                 activation='linear',  # type:str,
                 opt='adam',                              # optimizer name
                 lr=learning_rate,                                # learning rate
                 use_batch_norm=False,
                 use_layer_norm=False):
    #print('compiled_tcn\n')
    dilations = adjust_dilations(dilations)    # whether dilaitions are 2** or not / return dilations list

    input_layer = Input(shape=(max_len, num_feat))
    #print('input : ', input_layer)    # input shape : (None, 10000, 1)
    
    filters1 = 32
    filters2 = 64
    block = Conv1D(filters=filters1,
                   kernel_size=8,
                   padding='same',
                   activation='relu',
                   strides=2)(input_layer)
    block = MaxPooling1D(pool_size=3)(block)

       
    block = Conv1D(filters=filters2,
                   kernel_size=8,
                   padding='same',
                   activation='relu',
                   strides=2)(block)
    block = MaxPooling1D(pool_size=3)(block)
    
    '''
    block = Conv1D(filters=filters2,
                   kernel_size=8,
                   padding='same',
                   activation='relu',
                   strides=2)(block)
    block = MaxPooling1D(pool_size=3)(block)
    '''
    x = TCN(nb_filters, kernel_size, nb_stacks, dilations, padding,
            use_skip_connections, dropout_rate, return_sequences,
            activation, kernel_initializer, use_batch_norm, use_layer_norm,
            name=name)(block)
    

    #print('x.shape=', x.shape)   # x.shape : (None, nb_filters)

    def get_opt():
        if opt == 'adam':
            return optimizers.Adam(lr=lr, clipnorm=1.)
        elif opt == 'rmsprop':
            return optimizers.RMSprop(lr=lr, clipnorm=1.)
        else:
            raise Exception('Only Adam and RMSProp are available here')

    if not regression:
        print('[-] do classification...')
        #x = Dense(1024, activation='relu')(x)

        x = Dense(num_classes)(x)
        x = Activation('softmax')(x)
        output_layer = x
        model = Model(input_layer, output_layer)

        # https://github.com/keras-team/keras/pull/11373
        def accuracy(y_true, y_pred):
        # reshape in case it's in shape (num_samples, 1) instead of (num_samples,)
            if K.ndim(y_true) == K.ndim(y_pred):
                y_true = K.squeeze(y_true, -1)
                # convert dense predictions to labels
                y_pred_labels = K.argmax(y_pred, axis=-1)
                y_pred_labels = K.cast(y_pred_labels, K.floatx())
                return K.cast(K.equal(y_true, y_pred_labels), K.floatx())

        model.compile(get_opt(), loss='sparse_categorical_crossentropy', metrics=[accuracy])

    else:
        # regression
        x = Dense(output_len)(x)
        x = Activation('linear')(x)
        output_layer = x
        model = Model(input_layer, output_layer)
        model.compile(get_opt(), loss='mean_squared_error')
    print('model.x = {}'.format(input_layer.shape))
    print('model.y = {}'.format(output_layer.shape))
    return model


def tcn_full_summary(model, expand_residual_blocks=True):
    layers = model._layers.copy()  # store existing layers
    model._layers.clear()  # clear layers

    for i in range(len(layers)):
        if isinstance(layers[i], TCN):
            for layer in layers[i]._layers:
                if not isinstance(layer, ResidualBlock):
                    if not hasattr(layer, '__iter__'):
                        model._layers.append(layer)
                else:
                    if expand_residual_blocks:
                        for lyr in layer._layers:
                            if not hasattr(lyr, '__iter__'):
                                model._layers.append(lyr)
                    else:
                        model._layers.append(layer)
        else:
            model._layers.append(layers[i])

    model.summary()  # print summary

    # restore original layers
    model._layers.clear()
    [model._layers.append(lyr) for lyr in layers]

In [6]:
def do_tcn(tr_x, tr_y, te_x, te_y):
    model = compiled_tcn(return_sequences=False,
                         num_feat=1,
                         num_classes=5,
                         nb_filters=filters_num,
                         kernel_size=filter_size,
                         lr=learning_rate,
                         dilations=[2 ** i for i in range(dilation_num)],
                         nb_stacks=1,
                         max_len=tr_x[0:1].shape[1],
                         use_skip_connections=True)
    
    print(f'x_train.shape = {tr_x.shape}')
    print(f'y_train.shape = {tr_y.shape}')
    
    print(f'x_test.shape = {te_x.shape}')
    print(f'y_test.shape = {te_y.shape}')
    
    tcn_full_summary(model, expand_residual_blocks=False)
    
    #tr_y.squeeze().argmax(axis=1) shape : (2400,)
    
    
    with tf.device('/GPU:0'):
      model.fit(tr_x, tr_y.squeeze().argmax(axis=1), epochs=TRAIN_EPOCHS, batch_size=64,
                validation_data=(te_x, te_y.squeeze().argmax(axis=1)))
    
    model.evaluate(te_x, te_y.squeeze().argmax(axis=1))
    predictions = model.predict(te_x)
    #print(result.shape)   #(600, 5)

    # stat and save
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(te_y, axis=1)
    if len(predicted_labels) > len(true_labels):
        num_pad = len(predicted_labels) - len(true_labels)
        true_labels = np.concatenate([true_labels, true_labels[0:num_pad]])
    #print(len(predicted_labels))
    #print(len(true_labels))
    len_test = len(true_labels)
    cf_ma = np.zeros((5,5), dtype=int)
    #print(true_labels, predicted_labels)
    update_confusion_matrix(cf_ma, true_labels, predicted_labels)
    metrics_list = []
    for i in range(5):
        acc = truncate((float(len_test-cf_ma[:,i].sum()-cf_ma[i,:].sum()+cf_ma[i,i]*2)/len_test)*100, 4)
        metrics_list.append([class5[i], str(i), str(cf_ma[i,0]), str(cf_ma[i,1]), str(cf_ma[i,2]), str(cf_ma[i,3]), str(cf_ma[i,4]), str(acc)])
    overall_acc = truncate((float(cf_ma[0,0]+cf_ma[1,1]+cf_ma[2,2]+cf_ma[3,3]+cf_ma[4,4])/len_test)*100, 4)

    filename = 'test_result.txt'
    print('[*] Writing output text file...\n\n')
    with open(filename,'w') as f:
        f.write("\n")
        f.write('CLASS_NUM: 5\n')
        f.write('learning rate: ' + str(learning_rate) + "\n")
        f.write('TRAIN_EPOCHS: ' + str(TRAIN_EPOCHS) + "\n")
        f.write('filters_num: ' + str(filters_num) + "\n")
        f.write('filter_size: ' + str(filter_size) + "\n")
        f.write('dilation_num: ' + str(dilation_num) + "\n")
        f.write("label\tindex\t0\t1\t2\t3\t4\tACC\n")
        for metrics in metrics_list:
            f.write('\t'.join(metrics) + "\n")
        f.write('Overall accuracy: ' + str(overall_acc) + "\n")


def main():
    tr_x, tr_y, te_x, te_y = get_data()
    #print(tr_x.max(axis=0), te_x.max(axis=0)) # max 15
    do_tcn(tr_x, tr_y, te_x, te_y)
    
if __name__ == '__main__':
    main()

[*] load data
[-] done!
[-] do classification...
model.x = (None, 10000, 1)
model.y = (None, 5)
x_train.shape = (2400, 10000, 1)
y_train.shape = (2400, 5, 1)
x_test.shape = (600, 10000, 1)
y_test.shape = (600, 5, 1)
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10000, 1)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, 5000, 32)          288       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1666, 32)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 833, 64)           16448     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 277, 64)           0         
____________________________________